В этом уроке применим линейную регрессию на практике – попробуем предсказать стоимость машин и понять, от каких факторов зависит ценообразование на автомобили. Помимо этого узнаем, какие переменные важны для прогнозирования и насколько хорошо полученная модель описывает данные.

# Загрузите данные, проверьте типы, пропущенные значения.

Сколько пропущенных значений встретилось в датасете?

In [90]:
import pandas as pd
import numpy as no
import statsmodels.api as sm
import statsmodels.formula.api as smf

In [92]:
df_cars = pd.read_csv('cars.csv')
df_cars.head()

,car_ID,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,1,3,alfa-romero giulia,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,2,3,alfa-romero stelvio,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,3,1,alfa-romero Quadrifoglio,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,4,2,audi 100 ls,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,5,2,audi 100ls,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [94]:
df_cars.isnull().sum()

car_ID              0
symboling           0
CarName             0
fueltype            0
aspiration          0
doornumber          0
carbody             0
drivewheel          0
enginelocation      0
wheelbase           0
carlength           0
carwidth            0
carheight           0
curbweight          0
enginetype          0
cylindernumber      0
enginesize          0
fuelsystem          0
boreratio           0
stroke              0
compressionratio    0
horsepower          0
peakrpm             0
citympg             0
highwaympg          0
price               0
dtype: int64

Не встречено пропущенных значений, что довольно хорошо

# Сгенерируем первый признак.

Использовать полное название машины – не самый хороший вариант, поэтому создадим новый признак – марку автомобиля (company). Для этого используйте столбец CarName, разбейте значения ячеек по пробелу и запишите в колонку первый элемент. Например:

'audi 100 ls' → 'audi'

Сколько всего уникальных марок машины встречается в датасете? Столбец CarName с полным названием машины удалите из датасета, а также car_ID, они не пригодятся для дальнейшего анализа.

In [98]:
df_cars['CarName'] = df_cars.CarName.str.split().str[0]
df_cars = df_cars.drop('car_ID', axis=1)
df_cars.head()

,symboling,CarName,fueltype,aspiration,doornumber,carbody,drivewheel,enginelocation,wheelbase,carlength,...,enginesize,fuelsystem,boreratio,stroke,compressionratio,horsepower,peakrpm,citympg,highwaympg,price
0,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495.0
1,3,alfa-romero,gas,std,two,convertible,rwd,front,88.6,168.8,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500.0
2,1,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,171.2,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500.0
3,2,audi,gas,std,four,sedan,fwd,front,99.8,176.6,...,109,mpfi,3.19,3.40,10.0,102,5500,24,30,13950.0
4,2,audi,gas,std,four,sedan,4wd,front,99.4,176.6,...,136,mpfi,3.19,3.40,8.0,115,5500,18,22,17450.0


In [100]:
df_cars['CarName'].value_counts().count()

28

**Ответ:** всего 28 уникальных марок машин в датасете

# Теперь внимательнее посмотрите на уникальные значения company. Часть из них оказалась с ошибками!

'maxda' → 'mazda'

'Nissan' → 'nissan'

'porcshce' → 'porsche'

'toyouta' → 'toyota'

'vokswagen' & 'vw' → 'volkswagen'

Сколько уникальных производителей осталось в итоге?

In [104]:
df_cars.CarName.value_counts()

CarName
toyota         31
nissan         17
mazda          15
honda          13
mitsubishi     13
subaru         12
peugeot        11
volvo          11
volkswagen      9
dodge           9
buick           8
bmw             8
audi            7
plymouth        7
saab            6
isuzu           4
porsche         4
alfa-romero     3
chevrolet       3
jaguar          3
vw              2
maxda           2
renault         2
toyouta         1
vokswagen       1
Nissan          1
mercury         1
porcshce        1
Name: count, dtype: int64

In [106]:
df_cars = df_cars.query("CarName not in ('Nissan', 'porcshce', 'vw', 'vokswagen', 'toyouta', 'maxda')")
df_cars.CarName.value_counts().count()

22

**Ответ:** осталось 22 уникальных автопроизводителя

# Отлично! Чтобы не перегружать модель большим количеством предикторов, оставим только часть из них:

'company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower'
также не забыв про то, что мы предсказываем – 'price'.

После этого посчитайте корреляцию между price и другими переменными. Чему равна корреляция между price и horsepower? Ответ округлите до 2 знаков после точки.

In [122]:
df_cars = df_cars.rename(columns={'CarName' : 'company'})
df_cars_new = df_cars[['company', 'fueltype', 'aspiration','carbody', 'drivewheel', 'wheelbase', 'carlength','carwidth', 'curbweight', 'enginetype', 'cylindernumber', 'enginesize', 'boreratio','horsepower', 'price']]
df_cars_new.head(10)

,company,fueltype,aspiration,carbody,drivewheel,wheelbase,carlength,carwidth,curbweight,enginetype,cylindernumber,enginesize,boreratio,horsepower,price
0,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,13495.000
1,alfa-romero,gas,std,convertible,rwd,88.6,168.8,64.1,2548,dohc,four,130,3.47,111,16500.000
2,alfa-romero,gas,std,hatchback,rwd,94.5,171.2,65.5,2823,ohcv,six,152,2.68,154,16500.000
3,audi,gas,std,sedan,fwd,99.8,176.6,66.2,2337,ohc,four,109,3.19,102,13950.000
4,audi,gas,std,sedan,4wd,99.4,176.6,66.4,2824,ohc,five,136,3.19,115,17450.000
5,audi,gas,std,sedan,fwd,99.8,177.3,66.3,2507,ohc,five,136,3.19,110,15250.000
6,audi,gas,std,sedan,fwd,105.8,192.7,71.4,2844,ohc,five,136,3.19,110,17710.000
7,audi,gas,std,wagon,fwd,105.8,192.7,71.4,2954,ohc,five,136,3.19,110,18920.000
8,audi,gas,turbo,sedan,fwd,105.8,192.7,71.4,3086,ohc,five,131,3.13,140,23875.000
9,audi,gas,turbo,hatchback,4wd,99.5,178.2,67.9,3053,ohc,five,131,3.13,160,17859.167


In [124]:
df_cars_new.corr(numeric_only=True)

,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
wheelbase,1.000000,0.874502,0.794294,0.779521,0.581549,0.500375,0.368960,0.603003
carlength,0.874502,1.000000,0.840107,0.877083,0.686610,0.610150,0.559491,0.695764
carwidth,0.794294,0.840107,1.000000,0.868727,0.741670,0.560484,0.656177,0.773887
curbweight,0.779521,0.877083,0.868727,1.000000,0.851462,0.645432,0.753425,0.841843
enginesize,0.581549,0.686610,0.741670,0.851462,1.000000,0.573355,0.804739,0.871825
boreratio,0.500375,0.610150,0.560484,0.645432,0.573355,1.000000,0.561280,0.538311
horsepower,0.368960,0.559491,0.656177,0.753425,0.804739,0.561280,1.000000,0.800908
price,0.603003,0.695764,0.773887,0.841843,0.871825,0.538311,0.800908,1.000000


**Ответ:** корреляция между ценой и количеством лошадиных сил равна 0.80

# Последний шаг в подготовке данных: линейная регрессия в python не справляется с категориальными переменными (типом object в pandas), поэтому давайте применим pd.get_dummies(). Пример использования:

df_dummy = pd.get_dummies(data=cars[[список_столбцов_типа_object]], drop_first = True)

Не забудьте присоединить к полученному датасету столбцы с переменными других типов :) Также обратите внимание: мы должны использовать тот же набор колонок, который был на предыдущем шаге!

Сколько колонок теперь имеется в датасете?

In [128]:
df_cars_new.dtypes

company            object
fueltype           object
aspiration         object
carbody            object
drivewheel         object
wheelbase         float64
carlength         float64
carwidth          float64
curbweight          int64
enginetype         object
cylindernumber     object
enginesize          int64
boreratio         float64
horsepower          int64
price             float64
dtype: object

In [146]:
df_cars_dummy = pd.get_dummies(data=df_cars_new[['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 
                                                 'enginetype', 'cylindernumber']], drop_first=True)
cars_new = pd.concat([df_cars_dummy, df_cars_new], axis=1)
cars_new = cars_new.drop(columns=['company', 'fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginetype', 'cylindernumber'])
cars_new.head(10)

,company_audi,company_bmw,company_buick,company_chevrolet,company_dodge,company_honda,company_isuzu,company_jaguar,company_mazda,company_mercury,...,cylindernumber_twelve,cylindernumber_two,wheelbase,carlength,carwidth,curbweight,enginesize,boreratio,horsepower,price
0,False,False,False,False,False,False,False,False,False,False,...,False,False,88.6,168.8,64.1,2548,130,3.47,111,13495.000
1,False,False,False,False,False,False,False,False,False,False,...,False,False,88.6,168.8,64.1,2548,130,3.47,111,16500.000
2,False,False,False,False,False,False,False,False,False,False,...,False,False,94.5,171.2,65.5,2823,152,2.68,154,16500.000
3,True,False,False,False,False,False,False,False,False,False,...,False,False,99.8,176.6,66.2,2337,109,3.19,102,13950.000
4,True,False,False,False,False,False,False,False,False,False,...,False,False,99.4,176.6,66.4,2824,136,3.19,115,17450.000
5,True,False,False,False,False,False,False,False,False,False,...,False,False,99.8,177.3,66.3,2507,136,3.19,110,15250.000
6,True,False,False,False,False,False,False,False,False,False,...,False,False,105.8,192.7,71.4,2844,136,3.19,110,17710.000
7,True,False,False,False,False,False,False,False,False,False,...,False,False,105.8,192.7,71.4,2954,136,3.19,110,18920.000
8,True,False,False,False,False,False,False,False,False,False,...,False,False,105.8,192.7,71.4,3086,131,3.13,140,23875.000
9,True,False,False,False,False,False,False,False,False,False,...,False,False,99.5,178.2,67.9,3053,131,3.13,160,17859.167


In [148]:
cars_new.shape

(197, 49)

**Ответ:** 49 колонок

# Сначала построим небольшую модель всего с одним предиктором цены (price) – horsepower. Затем уже строим модель которая будет учитывать все признаки



In [152]:
X = cars_new.horsepower
Y = cars_new.price
X = sm.add_constant(X)
model = sm.OLS(Y, X)
results = model.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                  price   R-squared:                       0.641
Model:                            OLS   Adj. R-squared:                  0.640
Method:                 Least Squares   F-statistic:                     348.9
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           2.63e-45
Time:                        00:54:24   Log-Likelihood:                -1947.5
No. Observations:                 197   AIC:                             3899.
Df Residuals:                     195   BIC:                             3906.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const      -3693.3512    973.270     -3.795      0.000   -5612.838   -1773.865
horsepower   163.3031      8.743     18.678      0.000     146.060     180.546
==============================================================================
Omnibus:                       44.988   Durbin-Watson:                   0.788
Prob(Omnibus):                  0.000   Jarque-Bera (JB):               83.188
Skew:                           1.130   Prob(JB):                     8.63e-19
Kurtosis:                       5.243   Cond. No.                         318.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [156]:
results = smf.ols('Y ~ cars_new.horsepower + cars_new.wheelbase + cars_new.carlength + cars_new.carwidth + cars_new.curbweight + cars_new.enginesize + cars_new.boreratio', data=cars_new).fit()
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:                      Y   R-squared:                       0.823
Model:                            OLS   Adj. R-squared:                  0.816
Method:                 Least Squares   F-statistic:                     125.5
Date:                Wed, 12 Mar 2025   Prob (F-statistic):           1.53e-67
Time:                        01:15:15   Log-Likelihood:                -1878.0
No. Observations:                 197   AIC:                             3772.
Df Residuals:                     189   BIC:                             3798.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept           -4.396e+04   1